### DNN기반 회귀 모델 구현
- 데이터셋  : iris.csv
- 피쳐/속성 : 3개 (Sepal_Length, Sepal_Width, Petal_Length)
- 타겟/라벨 : 1개 (Petal_Width)
- 학습 방법 : 지도학습 > 회귀
- 알고리즘  : 인공신경망(ANN) -> MLP, DNN : 은닉층이 많은 구성
- 프레임워크 : Pytorch

[1] 모듈 로딩 및 데이터 준비 <hr>

In [64]:
# 모듈 로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# Data 및 시각화 관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [65]:
# 활용 패키지 버전 체크 ==> 사용자 정의함수로 구현하세요~
print(f'Pytorch v.{torch.__version__}')
print(f'pandas v.{pd.__version__}')

Pytorch v.2.4.1
pandas v.2.0.3


In [66]:
# 데이터 로딩
DATA_FILE = '../../../Data/iris.csv'

# CSV >>> DataFrame
irisDF = pd.read_csv(DATA_FILE, usecols=[0,1,2,3])

# 확인
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스 목적 : iris 데이터를 학습 및 추론 목적
- 클래스 이름 : IrisRegModel
- 부모 클래스 : nn.Module
- 매개   변수 : 층별 입출력 개수 고정하기 때문에 필요 없음
- 속성 / 필드 :
- 기능 / 역할 : \__init__() : 모델 구조 설정, forward() : 순방향 학습 <= 오버라이딩(overriding)

- 클래스 구조
    * 입력층 :
        * 입력 3개(피쳐)
        * 출력 10개(퍼셉트론/뉴런 10개 존재)
    * 은닉층 :
        * 입력 10개
        * 출력 30개(퍼셉트론/뉴런 30개 존재)
    * 출력층 :
        * 입력 30개
        * 출력 1개(너비)

- 손실함수/활성화함수
    * 클래스 형태 ==> nn.MSELoss, nn.ReLU ==> init() 메서드
    * 함수 형태 ==> torch.nn.functional 아래에 ==> forward() 메서드

In [67]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer = nn.Linear(3, 10)
        self.hidden_layer = nn.Linear(10, 30)
        self.out_layer = nn.Linear(30, 1)

    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y = self.in_layer(input_data)   # f1w1+f2w2+f3w3+b
        y = F.relu(y)                   # relu => y 값의 범위 : 0 <= y

        # 은닉층 : 10개의 숫자 값(>=0)
        y = self.hidden_layer(y)
        y = F.relu(y)

        # 출력층 : 30개의 숫자 값(>=0) 회귀이므로 바로 반환(return)
        return self.out_layer(y)

In [68]:
# 모델 인스턴스 생성
model = IrisRegModel()

print(model)

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


In [69]:
# 모델 사용
summary(model, input_size=(1000000,3))

Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1000000, 1]              --
├─Linear: 1-1                            [1000000, 10]             40
├─Linear: 1-2                            [1000000, 30]             330
├─Linear: 1-3                            [1000000, 1]              31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 401
Input size (MB): 12.00
Forward/backward pass size (MB): 328.00
Params size (MB): 0.00
Estimated Total Size (MB): 340.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐 개수 : 3개
- 타겟 개수 : 1개
- 클래스 이름 : IrisDataset
- 부모 클래스 : utils.data.Dataset
- 속성 필드 : featureDF, targetDF, n_rows, n_features
- 필수 메서드
    * \__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    * \__len__(self) : 데이터의 개수 반환
    * \__getitem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환

In [70]:
class IrisDataset(Dataset):
    def __init__(self, featureDF, targetDF):
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS = torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS = torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS

In [71]:
# 데이터셋 인스턴스 생성

# DataFrame에서 피쳐와 타겟 추출
featureDF = irisDF[irisDF.columns[:-1]] # 2D (150, 3)
targetDF = irisDF[irisDF.columns[-1:]] # 2D (150, 1)

# 커스텀 데이터셋 인스턴스
irisDS = IrisDataset(featureDF, targetDF)

[4] 학습 준비
- 학습 횟수 : EPOCH (처음부터 끝까지 학습하는 단위)
- 배치 크기 : BATCH_SIZE (한 번에 학습할 데이터셋 양)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 : GPU/CPU) 
- 학습률 : LR 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001 ~ 0.1

In [72]:
# 학습 진행 관련 설정
EPOCH = 1
BATCH_SIZE = 10
BATCH_CNT = irisDF.shape[0] // BATCH_SIZE
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001

print(f'BATCH_CNT : {BATCH_CNT}')

BATCH_CNT : 15


- 인스턴스/객체 : 모델, 데이터셋, 최적화 (, 손실함수, 성능지표)

In [77]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')
print(f'{type(y_train)} {type(y_test)} {type(y_val)}')

trainDS = IrisDataset(X_train, y_train)
valDS  = IrisDataset(X_val, y_val)
testDS = IrisDataset(X_test, y_test)

# 데이터로더 인스턴스
trainDL = DataLoader(trainDS, batch_size=BATCH_SIZE)

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [74]:
# [테스트] 데이터로더
for feature, target in trainDL:
    print(feature.shape, target.shape, sep='\n')
    break

torch.Size([10, 3])
torch.Size([10, 1])


In [75]:
# 최적화 인스턴스 => W, b 텐서 즉, model.parameters() 전달
optimizer = optim.Adam(model.parameters(), lr=LR)

# 손실함수 인스턴스 => 회귀, MSE, MAE, RMSE ....
regLoss = nn.MSELoss()

[5] 학습 진행

In [76]:
# 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY = [[],[]], [[],[]]
CNT = irisDS.n_rows / BATCH_SIZE

for epoch in range(EPOCH):
    # 학습 모드로 모델 설정
    model.train()

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0, 0

    for featureTS, targetTS in trainDL:
        # 학습 진행
        pred_y = model(featureTS)

        # 손실 계산
        loss = regLoss(pred_y, targetTS)
        loss_total += loss.item()

        # 성능평가 계산
        score = R2Score()(pred_y, targetTS)
        score_total += score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 에포크 당 검증기능
    # 모델 검증 모드 설정
    model.eval()

    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS = torch.FloatTensor(valDS.featureDF.values)
        val_targetTS = torch.FloatTensor(valDS.targetDF.values)

        # 추론/평가
        pred_test_y = model(val_featureTS)
        # 손실 계산
        loss_val = regLoss(pred_test_y, val_targetTS)
        # 성능 평가
        score_val = R2Score()(pred_test_y, val_targetTS)

    

    # 손실값과 성능평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)